In [1]:

from dotenv import load_dotenv
load_dotenv()
import os
import sys
sys.path.append(
    os.environ.get('WORK_DIR')) #type: ignore
sys.path.append(
    os.environ.get('DBASE_DIR')) #type: ignore
from dbase.DataAPI.ThetaData import * #type: ignore
from dbase.database.SQLHelpers import * #type: ignore
import pandas as pd
from EventDriven.data import HistoricTradeDataHandler
from EventDriven.event import *
from queue import Queue
from trade.backtester_.backtester_ import PTDataset, PTBacktester
import pandas_ta as ta
from trade.assets.Stock import Stock
from trade.backtester_.utils.WalkForwardUtils import prev_monday 
from trade.backtester_.strats import BBandsTrend2
from trade.backtester_.strats import MAStrat
import yfinance as yf
from datetime import datetime
from EventDriven.backtest import OptionSignalBacktest
from copy import deepcopy
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2


Console Logging & File Logging Can be configured using STREAM_LOG_LEVEL and FILE_LOG_LEVEL in environment variables.
Propagate to root logger can be set using PROPAGATE_TO_ROOT_LOGGER in environment variables.
Example:
STREAM_LOG_LEVEL = 'DEBUG'
FILE_LOG_LEVEL = 'INFO'
PROPAGATE_TO_ROOT_LOGGER = 'False'

2025-03-13 10:33:54 trade.helpers.Logging INFO: Logging Root Directory: /Users/chiemelienwanisobi/cloned_repos/QuantTools/logs
Using Proxy URL: http://18.232.166.224:5500/thetadata


In [2]:
thetadata_start = '2021-01-01'

In [264]:
import json
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
key = 10
with open(f'../input/profitable_weights_{key}.json', 'r') as f:
    weights = json.load(f)
ttrades__ = pd.read_csv(f'../input/profitable_trades_{key}.csv').iloc[:, 1:]
ttrades__ = ttrades__[ttrades__.Ticker == 'AAPL']
ttrades__

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration,Ticker
8,23.0,527.0,539.0,151.167239,147.029684,-95.163776,-0.027371,2023-02-07,2023-02-24,17 days,AAPL
9,22.0,545.0,708.0,154.328258,170.369995,352.918211,0.103946,2023-03-06,2023-10-26,234 days,AAPL
30,20.0,727.0,753.0,192.160221,187.149994,-100.204532,-0.026073,2023-11-22,2024-01-02,41 days,AAPL


In [265]:
trades_ = ttrades__.copy()
trades_

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration,Ticker
8,23.0,527.0,539.0,151.167239,147.029684,-95.163776,-0.027371,2023-02-07,2023-02-24,17 days,AAPL
9,22.0,545.0,708.0,154.328258,170.369995,352.918211,0.103946,2023-03-06,2023-10-26,234 days,AAPL
30,20.0,727.0,753.0,192.160221,187.149994,-100.204532,-0.026073,2023-11-22,2024-01-02,41 days,AAPL


In [266]:
symbol_list = trades_.Ticker.unique()
untraded_symbols = [s for s in weights.keys() if s not in trades_.Ticker.unique()]
for s in untraded_symbols:
    weights.pop(s)
pd.Series(weights).sort_values(ascending=False)

AAPL    0.077484
dtype: float64

In [267]:

from EventDriven.riskmanager import RiskManager, close_cache, spot_cache, chain_cache, oi_cache, LOOKBACKS, order_cache
from pandas.tseries.offsets import BDay

rm = RiskManager(None, None, 1000000)
rm.OrderPicker.liquidity_threshold = 2
rm.OrderPicker.lookback = 10
rm.OrderPicker.data_availability_threshold = 0.15
date, tick = '2023-07-05', 'AVGO'
date, tick = '2024-08-13', 'TSM'
start = (pd.to_datetime(date) - BDay(30)).strftime('%Y-%m-%d')
right = 'C'
order_settings = {'type': 'spread',
 'specifics': [{'direction': 'long',
   'rel_strike': .50,
   'dte': 210,
   'moneyness_width': 0.5},
  #  {'direction': 'short',
  # 'rel_strike': .60,
  # 'dte': 270,
  # 'moneyness_width': 0.35}
],
 'name': 'vertical_spread'}



order =  rm.OrderPicker.get_order(tick, date, right, 2, order_settings)
order

{'result': 'SUCCESSFUL',
 'data': {'long': ['TSM20250321C280'],
  'trade_id': '&L:TSM20250321C280',
  'close': 1.515}}

In [268]:
max_cash = {}
cash = 20_000
for s, w in weights.items():
    if w * cash > 200:
        max_cash[s] = 2
    elif w * cash > 100:
        max_cash[s] = 1
    else:
        max_cash[s] = 0.5
max_cash

{'AAPL': 2}

In [269]:
#Backtest class 
## Find a way to not always reinitialize the backtest class, when want to redo
from EventDriven.execution import exec_cache
evb_backtest = OptionSignalBacktest(trades_, initial_capital=cash)

In [270]:
evb_backtest.portfolio.initial_capital

20000.0

In [271]:
w_map = {x: w  * 0.75 for x, w in weights.items()} ## 75% of the weights for each stock
evb_backtest.portfolio.weight_map = w_map
evb_backtest.portfolio.weight_map
evb_backtest.portfolio.risk_manager.OrderPicker.liquidity_threshold = 100
evb_backtest.portfolio.risk_manager.OrderPicker.lookback = 10
evb_backtest.portfolio.risk_manager.OrderPicker.data_availability_threshold = 0.5
evb_backtest.portfolio.order_settings = {'type': 'naked',
 'specifics': [{'direction': 'long',
   'rel_strike': .850,
   'dte': 300,
   'moneyness_width': 0.35},
   {'direction': 'short',
  'rel_strike': .60,
  'dte': 300,
  'moneyness_width': 0.35}
],
 'name': 'vertical_spread'}

# evb_backtest.portfolio.order_settings = {'type': 'naked',
#  'specifics': [{'direction': 'long',
#    'rel_strike': .750,
#    'dte': 210,
#    'moneyness_width': 0.5},

# ],
#  'name': 'vertical_spread'}


evb_backtest.portfolio.max_contract_price = max_cash
evb_backtest.executor.commission_rate = 0.65/100
evb_backtest.executor.commission_rate
evb_backtest.executor.max_slippage_pct = 0.075
evb_backtest.portfolio.roll_map = 180
evb_backtest.portfolio.allocated_cash_map
evb_backtest.portfolio.max_contract_price
evb_backtest.portfolio.roll_map

{'AAPL': 180}

In [272]:
pd.options.display.max_rows = 15
pd.options.display.max_columns = 15
signals = evb_backtest.bars.signal_df
# series = signals.set_index('Date')['MSFT']
# series[series != 0]
signals

,Date,AAPL
0,2023-02-07,1
1,2023-02-08,0
2,2023-02-09,0
3,2023-02-10,0
4,2023-02-13,0
...,...,...
231,2023-12-27,0
232,2023-12-28,0
233,2023-12-29,0
234,2024-01-01,0


In [273]:
signals_df = deepcopy(signals).set_index('Date')
signals_df[signals_df!=-1].sum().sum()

3.0

## Problems:

- Current Problem: SELL Signal and Buy Signal are put right next to each other in the queue. Whereas, it is meant to be Sell Signal -> Order Event -> Fill Event -> Buy Signal -> Order Event -> Fill Event

Solution:
- Use a tuple of action ```python['CLOSE', 'OPEN']```
- Put first action into queue and return ffunctionality to backtester. Backtester then handles all corresponding sequence.
- Do the same for action two.

In [274]:
import cProfile
import pstats
import io

profiler = cProfile.Profile()
profiler.enable()
#run backtest

evb_backtest.run()
profiler.disable()
stream = io.StringIO()
stats = pstats.Stats(profiler, stream=stream).sort_stats('cumulative')

Processing event: MARKET
Processing event: SIGNAL
Buy Details
Position: {'long': ['AAPL20231215C200'], 'short': ['AAPL20231215C220'], 'trade_id': '&L:AAPL20231215C200&S:AAPL20231215C220', 'close': 1.9800000000000004}, Date: 2023-02-07, Signal: SignalEvent type:LONG, symbol=AAPL, date:2023-02-07 00:00:00, Order Settings=None, signal_id:AAPL20230207LONG
Max Contract Price: 2, Cash at Hand: 10.460310440873497
Order Quantity 5 Cash at Hand 10.460310440873497 Close 1.9800000000000004
Processing event: ORDER
Buy Order Position: {'long': ['AAPL20231215C200'], 'short': ['AAPL20231215C220'], 'trade_id': '&L:AAPL20231215C200&S:AAPL20231215C220', 'close': 1.9800000000000004} Price: 2.078172907084609 Quantity: 5 Datetime: 2023-02-07 00:00:00
Processing event: FILL
2023-02-07 00:00:00
AAPL BUY
Event queue is empty, processed 4 event(s)
Processing event: MARKET
Event queue is empty, processed 1 event(s)
Processing event: MARKET
Event queue is empty, processed 1 event(s)
Processing event: MARKET
Even

In [259]:
idx = 1
buys = [x  for x in exec_cache['fill'].values() if x.direction == 'BUY']
buy1 = buys[idx]
print("Buy")
print(buy1.datetime,buy1.fill_cost/buy1.quantity, buy1.market_value)

sells = [x  for x in exec_cache['fill'].values() if x.direction == 'SELL']
sell1 = sells[idx]
print("Sell")
print(sell1.datetime,sell1.fill_cost/sell1.quantity, sell1.market_value)



Buy
2023-07-24 00:00:00 2.0524030902941046 2394.2592280052786
Sell
2023-12-26 00:00:00 2.52451874599075 2979.047007793141


In [260]:
pd.options.display.max_rows = 100
trades = evb_backtest.portfolio._trades.copy()#
trades[['Positions','Ticker','PnL', 'ReturnPct', 'EntryPrice', 'ExitPrice', 'EntryMarketValue', 'ExitMarketValue', 'EntryTime', 'ExitTime', 'EntryPrice', 'ExitPrice', 'Quantity', 'SignalID', 'Duration']]
# len(trades['SignalID'].unique())

,Positions,Ticker,PnL,ReturnPct,EntryPrice,ExitPrice,EntryMarketValue,ExitMarketValue,EntryTime,ExitTime,EntryPrice,ExitPrice,Quantity,SignalID,Duration
0,&L:NVDA20240119C380&S:NVDA20240119C450,NVDA,242631.189024,16.995656,203.943869,3670.103712,14185.070814,256998.259838,2023-01-25,2023-07-24,203.943869,3670.103712,70,NVDA20230125LONG,180
1,&L:NVDA20240621C475&S:NVDA20240621C480,NVDA,55426.377979,0.230031,205.240309,252.451875,239425.922801,297904.700779,2023-07-24,2023-12-26,205.240309,252.451875,1174,NVDA20230125LONG,155
2,&L:NVDA20240920C780&S:NVDA20240920C800,NVDA,-83446.270868,-0.278079,154.442333,111.495205,297555.553517,219161.082649,2023-12-26,2024-01-02,154.442333,111.495205,1943,NVDA20230125LONG,7


In [261]:
evb_backtest.portfolio.plot_portfolio()

In [262]:
evb_backtest.portfolio.aggregate()

Start                                                    2023-01-25 00:00:00
End                                                      2024-01-02 00:00:00
Duration                                                   342 days 00:00:00
Exposure Time [%]                                                      100.0
Equity Final [$]                                                    234611.3
Equity Peak [$]                                                412034.066223
Return [%]                                                       1122.001987
Buy & Hold Return [%]                                             146.178132
CAGR [%]                                                         1346.035941
Volatility Ann. [%]                                               239.825422
Sharpe Ratio                                                        2.104287
Sortino Ratio                                                       4.768172
Skew                                                                3.022509

In [19]:
eq = evb_backtest.portfolio._equity.copy()
eq[eq.index >= '2023-07-20']

,NVDA,cash,commission,Total
datetime,,,,
2023-07-20,15892.276354,19068.409277,0.052,34960.685631
2023-07-21,15212.276354,19068.409277,0.052,34280.685631
2023-07-24,14512.266372,19068.409277,0.104,33580.675649
2023-07-25,14162.609947,19068.409277,1.170,33231.019223
2023-07-26,13752.609947,19068.409277,1.170,32821.019223
...,...,...,...,...
2023-12-27,21155.335090,19068.409277,4.771,40223.744367
2023-12-28,21420.335090,19068.409277,4.771,40488.744367
2023-12-29,21685.335090,19068.409277,4.771,40753.744367


In [20]:
#0	&L:NVDA20240119C380&S:NVDA20240119C450
price = (evb_backtest.portfolio.options_data['NVDA20240119C380'] - evb_backtest.portfolio.options_data['NVDA20240119C450']).copy()
price[price.index >= '2023-01-25']
price[price.index.isin( ['2023-01-25', '2023-07-24'])]

,Open,High,Low,Close,Volume,Bid_size,CloseBid,Ask_size,CloseAsk,Midpoint,Weighted_midpoint
Datetime,,,,,,,,,,,
2023-01-25,1.69,2.04,1.67,2.04,-5,-253,1.9,59,1.95,1.925,2.082202
2023-07-24,35.90,37.75,40.20,37.86,-183,127,36.7,94,39.80,38.250,38.037282


In [21]:

pd.set_option('display.max_rows', 10000)
evb_backtest.portfolio.get_all_positions()

,,long,short,trade_id,close,quantity,market_value
datetime,symbol,,,,,,
2023-01-25,NVDA,[NVDA20240119C380],[NVDA20240119C450],&L:NVDA20240119C380&S:NVDA20240119C450,770.0,4,770.0
2023-01-26,NVDA,[NVDA20240119C380],[NVDA20240119C450],&L:NVDA20240119C380&S:NVDA20240119C450,844.0,4,844.0
2023-01-27,NVDA,[NVDA20240119C380],[NVDA20240119C450],&L:NVDA20240119C380&S:NVDA20240119C450,976.0,4,976.0
2023-01-30,NVDA,[NVDA20240119C380],[NVDA20240119C450],&L:NVDA20240119C380&S:NVDA20240119C450,688.0,4,688.0
2023-01-31,NVDA,[NVDA20240119C380],[NVDA20240119C450],&L:NVDA20240119C380&S:NVDA20240119C450,730.0,4,730.0
2023-02-01,NVDA,[NVDA20240119C380],[NVDA20240119C450],&L:NVDA20240119C380&S:NVDA20240119C450,956.0,4,956.0
2023-02-02,NVDA,[NVDA20240119C380],[NVDA20240119C450],&L:NVDA20240119C380&S:NVDA20240119C450,1358.0,4,1358.0
2023-02-03,NVDA,[NVDA20240119C380],[NVDA20240119C450],&L:NVDA20240119C380&S:NVDA20240119C450,1244.0,4,1244.0
2023-02-06,NVDA,[NVDA20240119C380],[NVDA20240119C450],&L:NVDA20240119C380&S:NVDA20240119C450,1218.0,4,1218.0


In [22]:


""""
Why do these two return weird stuff after run?
    current_weighted_holdings
    current_positions

I can't reconcile the cost with the data (NVM, haha)

"""
# evb_backtest.portfolio.all_positions

'"\nWhy do these two return weird stuff after run?\n    current_weighted_holdings\n    current_positions\n\nI can\'t reconcile the cost with the data (NVM, haha)\n\n'

In [23]:
order_cache['2024-01-03']

KeyError: '2024-01-03'

## Extend for  get_port_stats
- Buy & Hold
- Dates
- Trades
- _strategy in Aggregate
- The function

In [214]:
evb_backtest.trades.to_csv(f'/Users/chiemelienwanisobi/cloned_repos/QuantTools/EventDriven/output/profitable_trades_options_{key}.csv')